In [44]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from datetime import datetime,timedelta
import plotly.express as px

In [45]:
def load_and_convert_data(file_path: str):
    """
    Load trade data from a feather file and convert it to a polars DataFrame.
    :param file_path: Path to the feather file containing options trades.
    :return: Data as a polars DataFrame.
    """
    dfPolar = pl.read_ipc(file_path)
    return dfPolar


def filter_by_strategy(df, strategy):
    """
    Filter trades based on a given strategy.
    :param df: The DataFrame containing options trades.
    :param strategy: The trading strategy to filter by.
    :return: Filtered DataFrame.
    """
    return df.filter(pl.col('strategy') == strategy)

def create_column_index(df):
    """
    Create a dictionary mapping column names to their indices.
    :param df: The DataFrame to map.
    :return: Dictionary of column names to indices.
    """
    return {col: idx for idx, col in enumerate(df.columns)}



In [46]:
def process_closes_expiries(open_positions, current_date, column_dict):
  """
    Identifies and removes expired trading positions from the open_positions dictionary based on the current date.

    This function iterates through the 'open_positions' dictionary, which tracks all currently open trading positions,
    keyed by a unique identifier. For each position, it checks whether the expiration date has passed relative to the current_date.
    If a position has expired (defined as the current date
    being at least one day past the expiration date), it is marked for removal. After identifying all expired positions,
    they are removed from the 'open_positions' dictionary.

    Parameters:
    - open_positions: A dictionary tracking the currently open positions. Each key includes the expiration date of the
      position, which is used to determine if the position has expired.
    - current_date: A datetime object representing the current date. This is used to compare against the expiration
      dates of the open positions to determine if they have expired.
    - column_dict: A dictionary mapping column names to their indices. This is included for consistency with related
      functions, but is not used within this function.

    Returns:
    None. The function modifies the 'open_positions' dictionary in place by removing expired positions.
  """
  expiredTrades = []


  for key in open_positions.keys():
        if key[1]:
          date = datetime.strptime(key[1], "%Y-%m-%d %H:%M:%S")
          if current_date - date >= timedelta(days=1):
            expiredTrades.append(key)

  for items in expiredTrades:
    open_positions.pop(items)

In [47]:
def handle_new_trade_position_opening(order, row, column_dict, open_positions, PnL):
    """
    Processes the opening of a trading position based on the provided trade details and adjusts the Profit and Loss (PnL).

    This function takes an order dictionary containing details about the trade action, quantity, and other relevant
    option information. It uses this information along with the current market data to update the 'open_positions' dictionary,
    which tracks all open trading positions. Depending on whether the action is 'selltoopen' or 'buytoopen',
    the PnL is adjusted accordingly. The 'column_dict' parameter is used to map column names
    to their indices in 'row', allowing for dynamic access to data fields.

    Parameters:
    - order: A dictionary containing details of the trade, including the action ('selltoopen' or 'buytoopen'),
      quantity, expiration date, strike price, and whether it's a call or put option.
    - row: A df row representing the current market data for the option.
    - column_dict: A dictionary mapping the column names to their indices in row.
    - open_positions: A dictionary that tracks currently open positions, keyed by a unique identifier for the option
      (comprising the underlying symbol, expiration date, strike price, and option type).
    - PnL: A float representing the current profit and loss, which will be adjusted by the function.

    Returns:
    - The updated PnL after processing the opening order.
    """

    action = order['m']['action']['s']
    quantity = int(order['m']['quantity']['n'])
    price = float(row[column_dict.get('price')])

    #uniquely identify each option by the given 4 values
    option_key = (row[column_dict.get('underlying_symbol')],
                  order['m']['expiration_date']['s'],
                  order['m']['strike_price']['n'],
                  order['m']['call_or_put']['s'])


    if action == 'selltoopen':
        PnL += price * quantity
    else: #buytoopen
        PnL -= price * quantity

    open_positions[option_key] = {
        'price': price,
        'underlying_price': row[column_dict.get('underlying_price')],
        'action': action,
        'quantity': quantity
    }
    return PnL

In [48]:
def handle_position_closing(order, row, column_dict, open_positions, PnL):
    """
    Processes the closing of an existing trading position based on the provided order details and adjusts the Profit
    and Loss (PnL).

    This function examines an order to close a position ('selltoclose' or 'buytoclose') and adjusts the position's
    quantity or removes it entirely from the 'open_positions' dictionary if the order quantity equals or exceeds
    the position's quantity. The PnL is adjusted based on the closing action, reflecting the realization of profit or
    loss as the position is reduced or closed. The function uses 'column_dict' to dynamically access the necessary
    data fields from 'row', facilitating flexibility in data structure.

    Parameters:
    - order: A dictionary containing details of the trade, including the action ('selltoclose' or 'buytoclose'),
      quantity, and other relevant option information.
    - row: A df row representing the current market data for the option.
    - column_dict: A dictionary mapping the column names to their indices in row.
    - open_positions: A dictionary that tracks currently open positions, keyed by a unique identifier for the option
      (comprising the underlying symbol, expiration date, strike price, and option type).
    - PnL: A float representing the current profit and loss, which will be adjusted by the function.

    Returns:
    - The updated PnL after processing the closing trade.
    """
    action = order['m']['action']['s']
    quantity = int(order['m']['quantity']['n'])
    price = float(row[column_dict.get('price')])

    option_key = (row[column_dict.get('underlying_symbol')],
                  order['m']['expiration_date']['s'],
                  order['m']['strike_price']['n'],
                  order['m']['call_or_put']['s'])

    #this checks if the trade is a valid trade or not
    # cause closing a option not present in the account is invalid
    if option_key in open_positions:

        if action == 'selltoclose':
            # checks how many options are sold
            PnL += price * min(quantity, open_positions[option_key]['quantity'])
        elif action == 'buytoclose':
            #checks how many options are bought to close
            PnL -= price * min(quantity, open_positions[option_key]['quantity'])

        #if the quantity of the options closed are less than the original quantity just subtract
        if quantity < open_positions[option_key]['quantity']:
            open_positions[option_key]['quantity'] -= quantity
        else:
            # else close the option position
            open_positions.pop(option_key)

    return PnL

In [49]:
def update_PnL_and_positions(order, row, column_dict, open_positions, PnL):
    """
    Updates the Profit and Loss (PnL) and the positions dictionary based on the given order. This function determines
    whether the order is for opening a new position or closing an existing one by checking the 'is_close' flag within
    the order trade.

     Parameters:
    - order: A dictionary containing details of the trade, including the action ('selltoclose' or 'buytoclose'),
      quantity, and other relevant option information.
    - row: A df row representing the current market data for the option.
    - column_dict: A dictionary mapping the column names to their indices in row.
    - open_positions: A dictionary that tracks currently open positions, keyed by a unique identifier for the option
      (comprising the underlying symbol, expiration date, strike price, and option type).
    - PnL: A float representing the current profit and loss, which will be adjusted by the function.

    Returns:
    - The updated PnL after processing the closing trade.

    """

    if not order['m']['is_close']['bool']:
        PnL = handle_new_trade_position_opening(order, row, column_dict, open_positions, PnL)
    else:
        PnL = handle_position_closing(order, row, column_dict, open_positions, PnL)
    return PnL

In [50]:
def process_trades(option_trades, column_dict):
    """
    Processes a sequence of option trades to compute the Profit and Loss (PnL) over time and manages open positions.

    This function iterates through a sorted collection of option trades, updating open positions and calculating PnL
    as it processes each trade. It handles the expiry of options based on their execution dates and updates PnL and
    positions for each trade within the option trade legs. The function assumes each row in option_trades represents
    a trade or a set of trades (order_legs) and uses the `executed_at` column to determine the current date for expiry checks.

    Parameters:
    - option_trades: A collection of option trades, expected to be a DataFrame or similar structure that can be sorted
      by the 'executed_at' column and supports iteration over its rows. Each row represents an option trade or a set of
      trades that occurred at a specific execution time.
    - column_dict: A dictionary mapping key column names (such as 'executed_at') to their indices or names in the
      `option_trades` structure, allowing for dynamic access to trade data.

    Returns:
    - PnL_total: A list of cumulative Profit and Loss values calculated after processing each trade.
    - PnL_time: A list of timestamps corresponding to each PnL calculation, reflecting the execution time of each
      trade or set of trades processed

    """

    option_trades = option_trades.sort('executed_at')
    PnL = 0
    open_positions = {}
    PnL_total = []
    PnL_time = []

    for row in option_trades.iter_rows():
        current_date = row[column_dict.get('executed_at')]

        process_closes_expiries(open_positions, current_date, column_dict)

        order_leg = row[-1]

        for order in order_leg:

            PnL = update_PnL_and_positions(order, row, column_dict, open_positions, PnL)

        PnL_total.append(PnL)
        PnL_time.append(current_date)



    return PnL_total, PnL_time

In [51]:
def pnLForOptions():
  """
    Loads option trades data from the file, processes the trades based on different option strategies, and
    visualizes the Profit and Loss (PnL) over time for each strategy.

    This function performs several key steps:
    - It loads option trade data from a feather file.
    - Filters the trades based on a list of option strategies.
    - Processes each set of trades filtered by strategy to compute PnL over time.
    - Visualizes the PnL for each option strategy over time using a line chart.

    The visualization part is assumed to be handled by Plotly, a plotting library that enables creating interactive
    charts.

    Parameters:
    None. The file path to the trade data is hardcoded within the function.

    Returns:
    None. The function's primary output is the visualization of PnL over time for various option strategies.
  """

  file_path = '/content/options_trades.feather'
  df = load_and_convert_data(file_path=file_path)


  option_strategy_list = ['Option', 'Straddle', 'Strangle', 'Covered', 'Vertical', 'Iron Condor', 'Ratio Spread', 'Calendar','Butterfly','Jade Lizard']

  print(df)
  print(df.columns)
  trade_data = {}
  for strat_name in option_strategy_list:
    trade_data[strat_name] = filter_by_strategy(df,strat_name)

  column_dict = create_column_index(df)



  for i in option_strategy_list:
      option_trades = trade_data[i]

      PnL_total, PnL_time = process_trades(option_trades, column_dict)

        # Visualization (assuming Plotly is preferred)
      fig = px.line(x=PnL_time, y=PnL_total, title=f'PnL for {i} Trades')
      fig.show()



pnLForOptions()

shape: (4_521, 25)
┌───────────┬──────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ strategy_ ┆ order_id ┆ underlyin ┆ strategy  ┆ … ┆ is_scalp_ ┆ is_earnin ┆ reason    ┆ order_leg │
│ id        ┆ ---      ┆ g_symbol  ┆ ---       ┆   ┆ trade     ┆ gs_play   ┆ ---       ┆ s         │
│ ---       ┆ str      ┆ ---       ┆ str       ┆   ┆ ---       ┆ ---       ┆ str       ┆ ---       │
│ str       ┆          ┆ str       ┆           ┆   ┆ str       ┆ str       ┆           ┆ list[stru │
│           ┆          ┆           ┆           ┆   ┆           ┆           ┆           ┆ ct[1]]    │
╞═══════════╪══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 12        ┆ 43171489 ┆ /ZWZ3     ┆ Option    ┆ … ┆ null      ┆ null      ┆ null      ┆ [{{{"./ZW │
│           ┆          ┆           ┆           ┆   ┆           ┆           ┆           ┆ Z3 OZWX3  │
│           ┆          ┆           ┆           ┆   ┆           ┆        

PNL Per Asset Class

In [52]:
# def update_PnL_per_asset(order, row, column_dict, open_positions, current_date, underlying_symbol, PnLDict):
#     """
#     Updates the Profit and Loss (PnL) and the positions dictionary based on the given order. This function determines
#     whether the order is for opening a new position or closing an existing one by checking the 'is_close' flag within
#     the order trade.

#      Parameters:
#     - order: A dictionary containing details of the trade, including the action ('selltoclose' or 'buytoclose'),
#       quantity, and other relevant option information.
#     - row: A df row representing the current market data for the option.
#     - column_dict: A dictionary mapping the column names to their indices in row.
#     - open_positions: A dictionary that tracks currently open positions, keyed by a unique identifier for the option
#       (comprising the underlying symbol, expiration date, strike price, and option type).
#     - PnL: A float representing the current profit and loss, which will be adjusted by the function.

#     Returns:
#     - The updated PnL Dict

#     """

#     PnL = 0

#     if underlying_symbol in PnLDict.keys():
#       length = len(PnLDict[underlying_symbol])
#       PnL = PnLDict[underlying_symbol][length-1][0]
#     else:
#       PnLDict[underlying_symbol] = []

#     if not order['m']['is_close']['bool']:
#         PnLDict[underlying_symbol].append((handle_new_trade_position_opening(order, row, column_dict, open_positions, PnL),current_date))
#     else:
#         PnLDict[underlying_symbol].append((handle_position_closing(order, row, column_dict, open_positions, PnL),current_date))
#     return PnLDict

In [53]:
def update_PnL_per_underlying_asset(order, row, column_dict, open_positions, current_date, underlying_symbol, PnLDict):
    """
    Updates the Profit and Loss (PnL) and the positions dictionary based on the given order. This function determines
    whether the order is for opening a new position or closing an existing one by checking the 'is_close' flag within
    the order trade.

     Parameters:
    - order: A dictionary containing details of the trade, including the action ('selltoclose' or 'buytoclose'),
      quantity, and other relevant option information.
    - row: A df row representing the current market data for the option.
    - column_dict: A dictionary mapping the column names to their indices in row.
    - open_positions: A dictionary that tracks currently open positions, keyed by a unique identifier for the option
      (comprising the underlying symbol, expiration date, strike price, and option type).
    - PnL: A float representing the current profit and loss, which will be adjusted by the function.

    Returns:
    - The updated PnL after processing the closing trade.

    """

    PnL = 0
    if underlying_symbol.startswith('/'):
      underlying_symbol = 'Futures'
    else:
      underlying_symbol = 'Stock'
    if underlying_symbol in PnLDict.keys():
      length = len(PnLDict[underlying_symbol])
      PnL = PnLDict[underlying_symbol][length-1][0]
    else:
      PnLDict[underlying_symbol] = []

    if not order['m']['is_close']['bool']:
        PnLDict[underlying_symbol].append((handle_new_trade_position_opening(order, row, column_dict, open_positions, PnL),current_date))
    else:
        PnLDict[underlying_symbol].append((handle_position_closing(order, row, column_dict, open_positions, PnL),current_date))
    return PnLDict

In [54]:
def process_trades_per_asset(option_trades, column_dict):
    """
    Processes a sequence of option trades to compute the Profit and Loss (PnL) over time and manages open positions.

    This function iterates through a sorted collection of option trades, updating open positions and calculating PnL DICT
    as it processes each trade. It handles the expiry of options based on their execution dates and updates PnL and
    positions for each trade within the option trade legs. The function assumes each row in option_trades represents
    a trade or a set of trades (order_legs) and uses the `executed_at` column to determine the current date for expiry checks.

    Parameters:
    - option_trades: A collection of option trades, expected to be a DataFrame or similar structure that can be sorted
      by the 'executed_at' column and supports iteration over its rows. Each row represents an option trade or a set of
      trades that occurred at a specific execution time.
    - column_dict: A dictionary mapping key column names (such as 'executed_at') to their indices or names in the
      `option_trades` structure, allowing for dynamic access to trade data.

    Returns:
    - PnL_total: A list of cumulative Profit and Loss values calculated after processing each trade.
    - PnL_time: A list of timestamps corresponding to each PnL calculation, reflecting the execution time of each
      trade or set of trades processed

    """

    option_trades = option_trades.sort('executed_at')
    PnLDict = {}
    open_positions = {}
    PnL_total = []
    PnL_time = []

    for row in option_trades.iter_rows():
        current_date = row[column_dict.get('executed_at')]
        underlying_symbol = row[column_dict.get('underlying_symbol')]

        process_closes_expiries(open_positions, current_date, column_dict)

        order_leg = row[-1]

        for order in order_leg:
            PnLDict = update_PnL_per_underlying_asset(order, row, column_dict, open_positions, current_date, underlying_symbol, PnLDict)

        PnL_time.append(current_date)



    return PnLDict, PnL_time

In [55]:

def calcPnLPerAsset():
  """
    Processes option trades data, computes Profit and Loss (PnL) for each option strategy per asset,
    and visualizes the PnL over time for each strategy per asset.

    Returns:
    None
  """
  file_path = 'options_trades.feather'
  df = load_and_convert_data(file_path=file_path)


  option_strategy_list = ['Option', 'Straddle', 'Strangle', 'Covered', 'Vertical', 'Iron Condor', 'Ratio Spread', 'Calendar','Butterfly','Jade Lizard']

  non_option_strategy_list = ['Crypto','Stock','Futures','Futures Spread']


  trade_data = {}
  for strat_name in option_strategy_list:
    trade_data[strat_name] = filter_by_strategy(df,strat_name)

  column_dict = create_column_index(df)



  for optionStrategy in option_strategy_list:
      option_trades = trade_data[optionStrategy]

      assetPnLMapping = {}

      PnLDict, PnLDates = process_trades_per_asset(option_trades, column_dict)



      length_total = len(PnLDates)


      for j in PnLDict.keys():
        final_PnL = np.full(length_total, np.nan)




        for k in PnLDict[j]:

          index = PnLDates.index(k[1])
          final_PnL[index] = k[0]

        assetPnLMapping[j] = final_PnL

        to_replace_value = -1
        for k in range(0,len(final_PnL)):
          if np.isnan(final_PnL[k]):
            if to_replace_value != -1:
              final_PnL[k] = to_replace_value
          else:
            to_replace_value = final_PnL[k]


      df_final = pd.DataFrame(assetPnLMapping)
      df_final['Time'] = PnLDates


      fig = px.line(df_final, x='Time', y=[df_final[j] for j in PnLDict.keys()], title=f'PnL for {optionStrategy} Trades based on each asset')

      fig.show()

calcPnLPerAsset()